In [28]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file = "./Resources/lending_data.csv"
df = pd.read_csv(file)

# Review the DataFrame
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df['loan_status']

# Separate the X variable, the features
X = df.drop(columns=['loan_status'])

In [6]:
# Review the y variable Series
y.sample(5)

40681    0
33862    0
23531    0
32048    0
8941     0
Name: loan_status, dtype: int64

In [8]:
# Review the X variable DataFrame
X.sample(5)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
23897,9000.0,6.936,45900,0.346405,3,0,15900
18199,8300.0,6.669,43400,0.308756,3,0,13400
38069,8300.0,6.663,43300,0.307159,2,0,13300
60439,10100.0,7.431,50500,0.405941,4,1,20500
42896,7800.0,6.438,41200,0.271845,2,0,11200


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [9]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [10]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [11]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
log_model = LogisticRegression(random_state=1)

# Fit the model using training data
log_model.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [12]:
# Make a prediction using the testing data
y_pred = log_model.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head()

,Prediction,Actual
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [14]:
# Print the balanced_accuracy score of the model
ba = balanced_accuracy_score(y_test, y_pred)
f'Balanced accuracy score is {round(ba,4)}'

'Balanced accuracy score is 0.9443'

In [18]:
# Generate a confusion matrix for the model
training_matrix = confusion_matrix(y_test, y_pred)
print(training_matrix)

[[18679    80]
 [   67   558]]


In [23]:
# Print the classification report for the model
training_report = classification_report(y_test, y_pred)
print(training_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logikstic regression model predicts 99% accuracy for both, and 87% for high risk loan.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [30]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
resample_model = RandomOverSampler(random_state = 1)

# Fit the original training data to the random_oversampler model
X_resample, y_resample = resample_model.fit_resample(X_train, y_train)

In [31]:
# Count the distinct values of the resampled labels data
X_resample.value_counts()
y_resample.value_counts()

0    56277
1    56277
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [33]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
resample_random_model = LogisticRegression(random_state = 1)

# Fit the model using the resampled training data
resample_random_model.fit(X_resample, y_resample)

# Make a prediction using the testing data
y_pred_resample = resample_random_model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [34]:
# Print the balanced_accuracy score of the model 
ba_resample= balanced_accuracy_score(y_test, y_pred_resample)
f'Balanced accuracy score is {round(ba_resample,4)}'

'Balanced accuracy score is 0.996'

In [35]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_test, y_pred_resample))

[[18668    91]
 [    2   623]]


In [36]:
# Print the classification report for the model
print(classification_report(y_test, y_pred_resample))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      1.00      0.93       625

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.96     19384
weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The model fiited with oversampled data had an accuracy score of 99% which is higher than the model fitted with imbalanced data. The model with oversampled data performs better because it does a better job in catching mistakes such as labeling high-risk loans as low-risk. This is analyzed based off of the recall score increasing from the imbalanced model to the oversampled model 0.91 - 0.99.